In [26]:
import tensorflow as tf
from datetime import datetime

def logdir():
    now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    root_logdir = 'tf_logs'
    _logdir = '{}/run-{}/'.format(root_logdir, now)
    return _logdir

def get_file_writer():
    return tf.summary.FileWriter(logdir(), tf.get_default_graph())

def draw_default_graph():
    get_file_writer().close()
    
class NewGraphDrawingContext:
    def __enter__(self):
        tf.reset_default_graph()
    def __exit__(self, type, value, traceback):
        get_file_writer().close()

In [28]:
with NewGraphDrawingContext():
    x = tf.Variable(3, name="x")
    y = tf.Variable(4, name="y")
    f = x * x * y + y + 2
    print(x, y, f)

<tf.Variable 'x:0' shape=() dtype=int32_ref> <tf.Variable 'y:0' shape=() dtype=int32_ref> Tensor("add_1:0", shape=(), dtype=int32)


In [30]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
print(sess.run(f))
sess.close()

42


In [31]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    print(f.eval())   

42


In [29]:
with NewGraphDrawingContext():
    x = tf.Variable(3, name="x")
    y = tf.Variable(4, name="y")
    f = x * x * y + y + 2
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        init.run()
        print(f.eval())

42


In [36]:
sess = tf.InteractiveSession()
x.initializer.run()
y.initializer.run()
print(f.eval())
sess.close()

42


In [38]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

True


In [40]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [41]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [42]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(sess.run([y, z]))

[10, 15]


In [34]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    print(theta.eval())

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [68]:
# boilerplate code
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

def show_default_graph():
    # Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
    # internal structure. We are going to visualize "Conv2D" nodes.
    tmp_def = rename_nodes(tf.get_default_graph().as_graph_def(), lambda s:"/".join(s.split('_',1)))
    show_graph(tmp_def)
    
show_default_graph()

In [39]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

N_EPOCHS = 1000
LEARNING_RATE = 0.01

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = preprocessing.scale(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - LEARNING_RATE * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        if epoch % 100 == 0:
            print("Epoch {} MSE = {}".format(epoch, mse.eval()))
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 9.165095329284668
Epoch 100 MSE = 4.986704349517822
Epoch 200 MSE = 4.923436164855957
Epoch 300 MSE = 4.89095401763916
Epoch 400 MSE = 4.867815017700195
Epoch 500 MSE = 4.8509297370910645
Epoch 600 MSE = 4.838573455810547
Epoch 700 MSE = 4.829500675201416
Epoch 800 MSE = 4.8228302001953125
Epoch 900 MSE = 4.8179097175598145
[[-0.38188982]
 [ 0.84309214]
 [ 0.15574653]
 [-0.22651283]
 [ 0.24508342]
 [ 0.00878068]
 [-0.04274311]
 [-0.61755705]
 [-0.58666176]]


In [41]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

N_EPOCHS = 1000
LEARNING_RATE = 0.01

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = preprocessing.scale(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - LEARNING_RATE * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        if epoch % 100 == 0:
            print("Epoch {} MSE = {}".format(epoch, mse.eval()))
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 7.023401737213135
Epoch 100 MSE = 4.898965358734131
Epoch 200 MSE = 4.821195125579834
Epoch 300 MSE = 4.812877655029297
Epoch 400 MSE = 4.810133457183838
Epoch 500 MSE = 4.808353900909424
Epoch 600 MSE = 4.807061672210693
Epoch 700 MSE = 4.806107521057129
Epoch 800 MSE = 4.8054022789001465
Epoch 900 MSE = 4.804877281188965
[[-8.7106228e-03]
 [ 8.3759958e-01]
 [ 1.3114499e-01]
 [-2.6022080e-01]
 [ 2.9234368e-01]
 [-9.9011617e-05]
 [-4.0544569e-02]
 [-8.0203676e-01]
 [-7.7263486e-01]]


In [43]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

N_EPOCHS = 1000
LEARNING_RATE = 0.01

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = preprocessing.scale(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        if epoch % 100 == 0:
            print("Epoch {} MSE = {}".format(epoch, mse.eval()))
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 8.2067232131958
Epoch 100 MSE = 4.948923110961914
Epoch 200 MSE = 4.875100135803223
Epoch 300 MSE = 4.854076862335205
Epoch 400 MSE = 4.840328216552734
Epoch 500 MSE = 4.8304219245910645
Epoch 600 MSE = 4.82321310043335
Epoch 700 MSE = 4.817950248718262
Epoch 800 MSE = 4.814104080200195
Epoch 900 MSE = 4.811290264129639
[[-0.25018597]
 [ 0.8299695 ]
 [ 0.1457097 ]
 [-0.21588877]
 [ 0.2425287 ]
 [ 0.00529879]
 [-0.04161403]
 [-0.70303416]
 [-0.6712967 ]]


In [53]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

tf.reset_default_graph()

N_EPOCHS = 1000
LEARNING_RATE = 0.01
BATCH_SIZE = 100

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
data_X = preprocessing.scale(housing_data_plus_bias)
data_y = housing.target.reshape(-1, 1)

def fetch_batch(epoch, batch_index, batch_size):
    X_batch = data_X[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    y_batch = data_y[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        for batch_index in range(m // BATCH_SIZE):
            X_batch, y_batch = fetch_batch(epoch, batch_index, BATCH_SIZE)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    print(best_theta)

KeyboardInterrupt: 

In [62]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

tf.reset_default_graph()

N_EPOCHS = 100
LEARNING_RATE = 0.01
BATCH_SIZE = 100

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
data_X = preprocessing.scale(housing_data_plus_bias)
data_y = housing.target.reshape(-1, 1)

def fetch_batch(epoch, batch_index, batch_size):
    X_batch = data_X[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    y_batch = data_y[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
        for batch_index in range(m // BATCH_SIZE):
            X_batch, y_batch = fetch_batch(epoch, batch_index, BATCH_SIZE)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")
    best_theta = theta.eval()
    print(best_theta)

[[-2.9345560e-01]
 [ 1.0048506e+00]
 [ 1.0344127e-01]
 [-3.6747351e-01]
 [ 3.5740542e-01]
 [-3.2015047e-03]
 [ 1.0916485e-03]
 [-2.7112114e-01]
 [-1.4902228e+00]]


In [71]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

tf.reset_default_graph()

N_EPOCHS = 100
LEARNING_RATE = 0.01
BATCH_SIZE = 100

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
data_X = preprocessing.scale(housing_data_plus_bias)
data_y = housing.target.reshape(-1, 1)

def fetch_batch(epoch, batch_index, batch_size):
    X_batch = data_X[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    y_batch = data_y[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    best_theta = theta.eval()
    print(best_theta)

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt
[[-2.9345560e-01]
 [ 1.0048506e+00]
 [ 1.0344127e-01]
 [-3.6747351e-01]
 [ 3.5740542e-01]
 [-3.2015047e-03]
 [ 1.0916485e-03]
 [-2.7112114e-01]
 [-1.4902228e+00]]


In [77]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

tf.reset_default_graph()

N_EPOCHS = 100
LEARNING_RATE = 0.01
BATCH_SIZE = 100

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
data_X = preprocessing.scale(housing_data_plus_bias)
data_y = housing.target.reshape(-1, 1)

def fetch_batch(epoch, batch_index, batch_size):
    X_batch = data_X[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    y_batch = data_y[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

file_writer = get_file_writer()
mse_summary = tf.summary.scalar('MSE', mse)

with tf.Session() as sess:
    init.run()
    
    for epoch in range(N_EPOCHS):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
        for batch_index in range(m // BATCH_SIZE):
            X_batch, y_batch = fetch_batch(epoch, batch_index, BATCH_SIZE)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * (m // BATCH_SIZE) + batch_index
                file_writer.add_summary(summary_str, step)
    
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")
    best_theta = theta.eval()
    print(best_theta)
    
file_writer.close()

[[-7.9301381e-01]
 [ 1.0048506e+00]
 [ 1.0344127e-01]
 [-3.6747351e-01]
 [ 3.5740542e-01]
 [-3.2015047e-03]
 [ 1.0916485e-03]
 [-2.7112114e-01]
 [-1.4902228e+00]]


In [79]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

with NewGraphDrawingContext():
    N_EPOCHS = 100
    LEARNING_RATE = 0.01
    BATCH_SIZE = 100

    housing = fetch_california_housing()
    m, n = housing.data.shape
    housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
    data_X = preprocessing.scale(housing_data_plus_bias)
    data_y = housing.target.reshape(-1, 1)

    def fetch_batch(epoch, batch_index, batch_size):
        X_batch = data_X[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
        y_batch = data_y[BATCH_SIZE * batch_index:BATCH_SIZE * (batch_index + 1)]
        return X_batch, y_batch

    X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
    y_pred = tf.matmul(X, theta, name="predictions")
    with tf.name_scope("loss") as scope:
        error = y_pred - y
        mse = tf.reduce_mean(tf.square(error), name="mse")
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
    training_op = optimizer.minimize(mse)

    init = tf.global_variables_initializer()

In [89]:
with NewGraphDrawingContext():

    def relu(X):
        with tf.variable_scope('relu', reuse=tf.AUTO_REUSE):
            threshold = tf.get_variable('threshold', shape=(),
                                        initializer=tf.constant_initializer(0.0))
            w_shape = (int(X.get_shape()[1]), 1)
            w = tf.Variable(tf.random_normal(w_shape), name='weights')
            b = tf.Variable(0.0, name='bias')
            z = tf.add(tf.matmul(X, w), b, name='z')
            return tf.maximum(z, threshold, name='relu')

    X = tf.placeholder(tf.float32, shape=(None, 3), name='X')
    relus = [relu(X) for _ in range(5)]
    output = tf.add_n(relus, name='output')
    
